# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

In [2]:
#Read Data
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [3]:
#Subset data to relevant field for analysis
data=data[['race','call']]

In [4]:
#Setup frequency table to get proportions
count_matrix = pd.crosstab(index=data["race"],columns=data["call"],margins=True)
count_matrix.columns = ["No Callback","Got Callback","rowtotal"]
count_matrix.index= ["b","w","coltotal"]
count_matrix

,No Callback,Got Callback,rowtotal
b,2278,157,2435
w,2200,235,2435
coltotal,4478,392,4870


In [5]:
#Get proportions adding to total candidates
count_matrix/count_matrix.ix["coltotal","rowtotal"]

,No Callback,Got Callback,rowtotal
b,0.467762,0.032238,0.5
w,0.451745,0.048255,0.5
coltotal,0.919507,0.080493,1.0


In [6]:
#Get proportions adding to row totals
percent_matrix=count_matrix.div(count_matrix["rowtotal"],axis=0)
percent_matrix

,No Callback,Got Callback,rowtotal
b,0.935524,0.064476,1.0
w,0.903491,0.096509,1.0
coltotal,0.919507,0.080493,1.0


### Statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

### Q1. What test is appropriate for this problem? Does CLT apply?

In [7]:
n1=count_matrix.iloc[0][2]
n2=count_matrix.iloc[1][2]
p1=percent_matrix.iloc[0][1]
p2=percent_matrix.iloc[1][1]

In [8]:
#Test Conditions
print(n1*p1,n2*p2)
print(n1*(1-p1),n2*(1-p2))

157.0 235.0
2278.0 2200.0


In [9]:
count_matrix

,No Callback,Got Callback,rowtotal
b,2278,157,2435
w,2200,235,2435
coltotal,4478,392,4870


In [10]:
percent_matrix

,No Callback,Got Callback,rowtotal
b,0.935524,0.064476,1.0
w,0.903491,0.096509,1.0
coltotal,0.919507,0.080493,1.0


### Q2. What are the null and alternate hypotheses?

### Q3. Compute margin of error, confidence interval, and p-value.

In [11]:
#Using 95% confidence interval 
pbar=((p1 * n1)+ (p2 * n2)) / (n1 + n2)
#Calculate Margin of Error
SE=math.sqrt(pbar*(1-pbar)*((1/n1)+(1/n2)))
#Calculate z-statistic
zstatistic=(p1-p2)/SE

In [12]:
zstatistic

-4.1084121524343464

In [13]:
#Calculate p-value
p_value = stats.norm.sf(abs(zstatistic))*2
p_value

3.9838868375850767e-05

In [14]:
#Confidence Interval
CIUpper=((p1-p2)+(stats.norm.ppf(0.975)*(math.sqrt(((p1*(1-p1))/n1)+((p2*(1-p2))/n2)))))
CILower=((p1-p2)-(stats.norm.ppf(0.975)*(math.sqrt(((p1*(1-p1))/n1)+((p2*(1-p2))/n2)))))
print("CI : (",CIUpper,",",CILower,")")

CI : ( -0.0167777281812 , -0.0472879802377 )


### Q4. Write a story describing the statistical significance in the context of the original problem.

### Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?